In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

if os.getcwd().split("/")[-1] == "sandbox":
    os.chdir("../")
    
load_dotenv()

True

In [2]:
from food_co2_estimator.chains.recipe_extractor import get_recipe_extractor_chain
from food_co2_estimator.utils.url import get_url_text
recipe_extractor = get_recipe_extractor_chain(verbose=False)

In [3]:
from food_co2_estimator.utils.url import get_url_text

from food_co2_estimator.output_parsers.recipe_extractor import Recipe


url = "https://www.valdemarsro.dk/frikadeller/"
text = get_url_text(url)
recipe_extractor = get_recipe_extractor_chain(verbose=False)
recipe: Recipe = recipe_extractor.invoke(input=text)

In [4]:
from food_co2_estimator.chains.rag_co2_estimator import rag_co2_emission_chain
from food_co2_estimator.chains.translator import get_translation_chain

In [5]:
# emission_chain = rag_co2_emission_chain(verbose=True)
# translation_chain = get_translation_chain()

# emission_chain_with_translation = translation_chain | emission_chain

# emissions = emission_chain_with_translation.invoke(
#     input={"inputs": recipe.ingredients, "language": "da"}
# )

In [14]:
from food_co2_estimator.retrievers.emission_retriever import get_emission_retriever_chain, remove_quantities_from_inputs

retriever = get_emission_retriever_chain(k=10)
cleaned = remove_quantities_from_inputs(["1 kg eggs"])
retriever.invoke(cleaned[0])

{'Eggs, chicken, free-range hens (indoor), raw': '1.0 kg CO2e / kg',
 'Average chicken': '3.3 kg CO2e / kg',
 'Noodles, eggs': '1.7 kg CO2e / kg',
 'Egg salad': '1.5 kg CO2e / kg'}

In [13]:
cleaned

['eggs']

In [8]:
from food_co2_estimator.language.translator import MyTranslator


translator = MyTranslator.default()
translator.translate("basmati ri")

'basmati rice'

In [9]:
from food_co2_estimator.data.vector_store import get_vector_store


vector_store = get_vector_store()

In [10]:
mmr_retriever = vector_store.as_retriever()
mmr_retriever.invoke("1 egg")

[Document(metadata={'DSK Kategori': 'Tilberedte/konserverede fødevarer', 'Detail': 0.0002392037588851976, 'Emballage': 0.2419647285623749, 'Energi (KJ/100 g)': 982.0, 'Fedt (g/100 g)': 22.0, 'Forarbejdning': 0.5753931404648314, 'ID_Ra': 'Ra00390', 'ID_food': 'Fa0191', 'ID_pack': 'Pa017', 'ID_retail': 'Ma0333', 'ILUC': 0.04472225069718327, 'Kategori': 'Smagsgivere og krydderier', 'Kulhydrat (g/100 g)': 3.7, 'Landbrug': 0.5147958677654496, 'Navn': 'Æggesalat', 'Produkt': 'Æggesalat', 'Protein (g/100 g)': 6.3, 'Total kg CO2e/kg': 1.50049349492172, 'Transport': 0.1233783036729955}, page_content='Egg salad'),
 Document(metadata={'DSK Kategori': 'Tilberedte/konserverede fødevarer', 'Detail': 0.00626463674608679, 'Emballage': 0.2330458035131551, 'Energi (KJ/100 g)': 1477.0, 'Fedt (g/100 g)': 2.1, 'Forarbejdning': 0.1661957753185934, 'ID_Ra': 'Ra00470', 'ID_food': 'Fa0007', 'ID_pack': 'Pa002', 'ID_retail': 'Ma0332', 'ILUC': 0.09065676623320963, 'Kategori': 'Korn og kornprodukter', 'Kulhydrat (